# Day 8

## Part 1

In [1]:
f = open("./data/input8.txt", "r")
c = f.readlines()


In [2]:
import pandas as pd
df = pd.DataFrame(c)
    

In [19]:
tmp = df[0].str.split('|').apply(pd.Series)
input = tmp[0]
output = tmp[1]

In [20]:
input = input.str.split(' ').apply(pd.Series).drop(10, axis=1)
output = output.str.split(' ').apply(pd.Series).drop(0, axis=1)
output[4] = output[4].str.replace('\n','')

In [30]:
res = []
for col in input.columns:
    res.append(input[col].str.len())
lengths = pd.concat(res, axis=1)

In [31]:
4 - has length 4
1 - has length 2
7 - has length 3
8 - has length 7

,0,1,2,3,4,5,6,7,8,9
0,6,6,4,5,6,2,5,5,7,3
1,7,3,5,6,6,6,5,5,4,2
2,7,4,6,3,6,5,5,5,2,6
3,4,6,5,6,2,7,6,5,3,5
4,6,5,2,5,7,6,3,6,5,4
...,...,...,...,...,...,...,...,...,...,...
195,2,6,5,7,3,6,5,6,5,4
196,5,4,6,6,5,7,3,6,5,2
197,5,3,6,6,6,4,5,5,2,7
198,6,5,7,6,4,3,6,2,5,5


In [51]:
res = []
for col in output.columns:
    res.append(output[col].str.len())
output_lengths = pd.concat(res, axis=1)

In [53]:
(output_lengths==4).sum().sum() + (output_lengths==2).sum().sum() +  (output_lengths==3).sum().sum() +  (output_lengths==7).sum().sum()

303

## Part 2

In [98]:
def get_digit(dict, str):
   
    if len(str) == 5:
        if len(set(dict[1])&set(str))==2:
            dict[3] = str
        elif len(set(dict[4])& set(str))==3:
            dict[5] = str
        else:
            dict[2]=str
        return dict
    elif len(str) == 6:
        if len(set(dict[7])& set(str))==2:
            dict[6] = str
        elif len(set(dict[4])&set(str))==4:
            dict[9]=str
        else:
            dict[0]=str
        return dict
    else:
        return dict

        
        

In [106]:
def find_digits(vv):
    dict = {}
    dict[1] = [x for x in vv if len(x)==2][0]
    dict[4] = [x for x in vv if len(x)==4][0]
    dict[7] = [x for x in vv if len(x)==3][0]
    dict[8] = [x for x in vv if len(x)==7][0]

    for v in vv:
        digits = get_digit(dict, v)
    return digits

digit_finder = lambda x: int(''.join([str(inv_dig[''.join(sorted(y))]) for y in x]))


In [111]:
import numpy as np
final_arr = []
for i in np.arange(input.shape[0]):
    digits = find_digits(input.loc[i].to_list())
    inv_dig = {''.join(sorted(v)): k for k, v in digits.items()}
    number = digit_finder(output.loc[i].to_list())
    final_arr.append(number)

In [110]:
np.array(final_arr).sum()


961734

# Day 9

## Part 1

In [1]:
import pandas as pd
import numpy as np

In [2]:
f = open("./data/input9.txt", "r")
c = f.readlines()
df = pd.DataFrame(c)
df = df[0].apply(lambda x: list(x)).apply(pd.Series).drop(100, axis=1)
df = df.astype(int)
df_t = df.T



In [3]:
a = df < df.shift(1).fillna(10)
b = df < df.shift(-1).fillna(10)
c = (df_t < df_t.shift(1).fillna(10)).T
d = (df_t < df_t.shift(-1).fillna(10)).T


In [4]:
df[(a==True)&(b==True)&(c==True)&(d==True)].sum().sum(), df[(a==True)&(b==True)&(c==True)&(d==True)].count().sum()


(276.0, 224)

In [5]:
276+224

500

## Part 2

In [6]:
arr = df[(a==True)&(b==True)&(c==True)&(d==True)].fillna(999).to_numpy()
low_points = np.where(arr!=999)

In [7]:
x = low_points[0]
y = low_points[1]
low_points = pd.DataFrame(zip(x,y))

In [57]:
def discover_basin(basin, df, x, y):
    
    # search right
    if df.loc[x,y]==9:
        return basin
    else:
        # search right
        if y+1 <df.shape[1]:
            if (df.loc[x,y] < df.loc[x,y+1]) :
                basin = discover_basin(basin, df, x, y+1)
                basin.append((x,y))
        # search left
        if y >0 :
            if (df.loc[x,y] < df.loc[x,y-1]):
                
                basin = discover_basin(basin, df, x, y-1)
                basin.append((x,y))

        # search down
        if x+1 < df.shape[0]:
            if (df.loc[x,y] < df.loc[x+1,y]):
                
                basin = discover_basin(basin, df, x+1, y)
                basin.append((x,y))
        if x>0:
                # search up
            if (df.loc[x,y] < df.loc[x-1,y]):
                
                basin = discover_basin(basin, df, x-1, y)
                basin.append((x,y))
        return basin
        
    
    

In [58]:
all_basins = []

for _, row in low_points.iterrows():
    basin = []
    basin = discover_basin(basin, df, row[0], row[1])
    all_basins.append(len(list(set(basin))))

In [59]:
pd.Series(all_basins).sort_values(ascending=False).head(3)

96     97
132    97
41     96
dtype: int64

In [71]:
basin = []
pd.Series(list(set(discover_basin(basin, df, 0, 7)))).apply(pd.Series).sort_values(by=[1,0])

,0,1
10,0,5
7,0,6
11,1,6
0,0,7
8,1,7
3,2,7
4,3,7
12,0,8
5,1,8
14,2,8


In [72]:
98*98*97

931588

In [65]:
low_points.head(15)

,0,1
0,0,7
1,0,17
2,0,26
3,0,37
4,0,48
5,0,56
6,0,64
7,1,41
8,1,86
9,1,95


In [34]:
row = (54,3)
basin = []
basin_down = []
basin_up = []

right_basin = discover_basin_right(basin, df,row[0], row[1])
horizontal_basin = discover_basin_left(right_basin, df, row[0], row[1])
horizontal_basin = list(set(horizontal_basin))

for point in horizontal_basin:
    copy = horizontal_basin.copy()
    basin_down.extend(discover_basin_down(copy, df, point[0], point[1]))
    copy = basin_down.copy()
    basin_up.extend(discover_basin_up(copy, df, point[0], point[1]))
    
horizontal_basin = set(horizontal_basin)
horizontal_basin.update(basin_down)
horizontal_basin.update(basin_up)
len(list(horizontal_basin))



46

In [35]:
len(list(set(basin_up)))

46

In [36]:
list(set(basin_up))


[(54, 7),
 (53, 5),
 (55, 2),
 (56, 4),
 (56, 1),
 (58, 1),
 (58, 4),
 (51, 1),
 (50, 3),
 (51, 4),
 (52, 3),
 (57, 2),
 (49, 4),
 (53, 1),
 (53, 4),
 (55, 1),
 (54, 3),
 (54, 9),
 (53, 7),
 (55, 4),
 (54, 6),
 (56, 3),
 (58, 3),
 (51, 3),
 (52, 5),
 (57, 1),
 (57, 4),
 (49, 3),
 (54, 2),
 (55, 3),
 (54, 5),
 (53, 3),
 (56, 2),
 (54, 8),
 (59, 4),
 (50, 4),
 (52, 1),
 (52, 4),
 (52, 7),
 (51, 5),
 (57, 3),
 (48, 3),
 (54, 4),
 (53, 2),
 (53, 8),
 (54, 1)]